In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df2023_2024 = pd.read_excel("DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx")

In [3]:
df2023_2024.head()

,Unnamed: 0,SURESTE,SURESTE.1,SURESTE.2,SURESTE.3,SURESTE.4,SURESTE.5,SURESTE.6,SURESTE.7,SURESTE.8,...,NOROESTE 3.5,NOROESTE 3.6,NOROESTE 3.7,NOROESTE 3.8,NOROESTE 3.9,NOROESTE 3.10,NOROESTE 3.11,NOROESTE 3.12,NOROESTE 3.13,NOROESTE 3.14
0,NaN,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,...,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
1,date,ppm,ppb,ppb,ppb,ppb,ug/m3,ug/m3,mmhg,mm/hr,...,ug/m3,ug/m3,mmhg,mm/hr,%,ppb,KW/m2,degC,KMPH,DEG
2,2023-01-01 00:00:00,2.37,54.5,32.6,87.1,3,110,68,721.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01 01:00:00,2.12,38.7,30.3,68.9,3,116,67.18,721.5,0,...,246,NaN,707.4,0,59,2.9,0,14.14,65.3,95
4,2023-01-01 02:00:00,2.05,38.7,28.8,67.4,3,117,75.12,721.1,0,...,239,NaN,707,0,59,3.2,0,14.11,60.9,90


In [4]:
#Select the column name until a . appears
df2023_2024.columns = df2023_2024.columns.str.split('.').str[0]
df2023_2024.columns

Index(['Unnamed: 0', 'SURESTE', 'SURESTE', 'SURESTE', 'SURESTE', 'SURESTE',
       'SURESTE', 'SURESTE', 'SURESTE', 'SURESTE',
       ...
       'NOROESTE 3', 'NOROESTE 3', 'NOROESTE 3', 'NOROESTE 3', 'NOROESTE 3',
       'NOROESTE 3', 'NOROESTE 3', 'NOROESTE 3', 'NOROESTE 3', 'NOROESTE 3'],
      dtype='object', length=240)

In [5]:
df2023_2024.columns = df2023_2024.columns + " " + df2023_2024.loc[0] + " (" + df2023_2024.loc[1] +")"
df2023_2024.drop([0,1], inplace=True)

In [6]:
df2023_2024.head()

,NaN,SURESTE CO (ppm),SURESTE NO (ppb),SURESTE NO2 (ppb),SURESTE NOX (ppb),SURESTE O3 (ppb),SURESTE PM10 (ug/m3),SURESTE PM2.5 (ug/m3),SURESTE PRS (mmhg),SURESTE RAINF (mm/hr),...,NOROESTE 3 PM10 (ug/m3),NOROESTE 3 PM2.5 (ug/m3),NOROESTE 3 PRS (mmhg),NOROESTE 3 RAINF (mm/hr),NOROESTE 3 RH (%),NOROESTE 3 SO2 (ppb),NOROESTE 3 SR (KW/m2),NOROESTE 3 TOUT (degC),NOROESTE 3 WSR (KMPH),NOROESTE 3 WDR (DEG)
2,2023-01-01 00:00:00,2.37,54.5,32.6,87.1,3,110,68,721.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01 01:00:00,2.12,38.7,30.3,68.9,3,116,67.18,721.5,0,...,246,NaN,707.4,0,59,2.9,0,14.14,65.3,95
4,2023-01-01 02:00:00,2.05,38.7,28.8,67.4,3,117,75.12,721.1,0,...,239,NaN,707,0,59,3.2,0,14.11,60.9,90
5,2023-01-01 03:00:00,2.5,60.5,29.1,89.4,3,135,82.81,720.8,0,...,275,NaN,706.8,0,59,2.3,0,14.08,58.1,84
6,2023-01-01 04:00:00,1.94,42.3,25.7,67.7,NaN,132,59.56,720.7,0,...,NaN,NaN,706.8,0,58,1.8,0,12.94,61.4,87


In [7]:
#Extract each sheet from the excel file
df2022_2023 = pd.read_excel("DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx", sheet_name=None)

In [8]:
df2022_2023.keys()

dict_keys(['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'SUROESTE2', 'SURESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE2', 'NORESTE3', 'NOROESTE3', 'CATÁLOGO'])

In [9]:
#Access to SURESTE sheet
df2022_2023["SURESTE"].head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
0,2022-01-01 00:00:00,2.36,19.8,48.4,68.0,5.0,142.0,111.00,715.7,0.0,59.0,4.0,0.0,21.65,6.5,285.0
1,2022-01-01 01:00:00,2.29,10.8,44.5,55.1,6.0,168.0,112.16,715.5,0.0,59.0,4.6,0.0,21.21,7.1,284.0
2,2022-01-01 02:00:00,2.71,28.3,47.1,75.2,4.0,203.0,139.31,715.4,0.0,62.0,5.9,0.0,21.21,7.7,290.0
3,2022-01-01 03:00:00,2.31,19.6,42.1,61.5,5.0,262.0,177.68,715.1,0.0,59.0,5.5,0.0,20.44,8.2,294.0
4,2022-01-01 04:00:00,1.85,13.0,39.5,52.5,4.0,185.0,134.21,715.1,0.0,59.0,4.9,0.0,19.80,8.1,287.0


In [10]:
df2020_2021 = pd.read_excel("DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx", sheet_name=None)


In [11]:
df2020_2021.keys()

dict_keys(['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'SUROESTE2', 'SURESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE2', 'NORESTE3', 'NOROESTE3', 'CATÁLOGO'])

In [12]:
df2020_2021["SURESTE"].head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,66.0,54.23,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
1,2020-01-01 01:00:00,2.11,NaN,NaN,NaN,19.0,57.0,NaN,735.7,0.0,96.0,5.4,0.01,11.20,8.1,NaN
2,2020-01-01 02:00:00,2.06,NaN,NaN,NaN,19.0,68.0,53.84,734.8,0.0,96.0,5.5,0.01,11.26,5.5,NaN
3,2020-01-01 03:00:00,1.96,NaN,NaN,NaN,19.0,68.0,36.47,734.2,0.0,96.0,5.4,0.01,11.35,3.8,NaN
4,2020-01-01 04:00:00,1.98,NaN,NaN,NaN,16.0,48.0,33.59,733.9,0.0,96.0,5.5,0.01,11.47,3.3,NaN


In [13]:
df2021_contaminante = pd.read_excel("DATOS HISTÓRICOS 2021_Contaminante.xlsx")


In [14]:
#Select the column name until a . appears
df2021_contaminante.columns = df2021_contaminante.columns.str.split('.').str[0]
df2021_contaminante.columns = df2021_contaminante.columns + " " + df2021_contaminante.loc[0] + " (" + df2021_contaminante.loc[1] +")"
df2021_contaminante.drop([0,1], inplace=True)

In [15]:
df2021_contaminante.head()

,NaN,NaN,SURESTE CO (ppm),SURESTE NO (ppb),SURESTE NO2 (ppb),SURESTE NOX (ppb),SURESTE O3 (ppb),SURESTE PM10 (ug/m3),SURESTE PM2.5 (ug/m3),SURESTE PRS (mmhg),...,NORTE 2 PRS (mmhg),NORTE 2 RAINF (mm/hr),NORTE 2 RH (%),NORTE 2 SO2 (ppb),NORTE 2 SR (KW/m2),NORTE 2 TOUT (degC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NaN,NaN
2,2021-01-01 00:00:00,00:00:00,3.02,NaN,NaN,NaN,9,111.86,79.19,NaN,...,NaN,NaN,69,NaN,0,8.18,5.8,113,NaN,NaN
3,2021-01-01 00:00:00,01:00:00,2.94,NaN,NaN,NaN,13,111.03,81.46,NaN,...,711.3,0,72,NaN,0,7.04,6,118,NaN,NaN
4,2021-01-01 00:00:00,02:00:00,2.94,NaN,NaN,NaN,12,68.7,48.07,NaN,...,711.6,0,76,NaN,0,5.98,6.1,103,NaN,NaN
5,2021-01-01 00:00:00,03:00:00,2.97,NaN,NaN,NaN,11,63.77,43.35,NaN,...,711.9,0,77,NaN,0,5.52,3.7,70,NaN,NaN
6,2021-01-01 00:00:00,04:00:00,3,NaN,NaN,NaN,11,72.11,45.47,NaN,...,712.2,0,79,NaN,0,4.95,3,93,NaN,NaN


In [16]:
df2020_contaminante = pd.read_excel("DATOS HISTÓRICOS 2020_Contaminante.xlsx", sheet_name=None)


In [17]:
df2020_contaminante.keys()

dict_keys(['PM10', 'PM2.5', 'O3', 'NO', 'NO2', 'NOx', 'CO', 'SO2'])

In [18]:
df2020_contaminante["O3"].head()

,parameter,date,SE,SE_b,NE,NE_b,CE,CE_b,NO,NO_b,...,SO2,SO2_b,SE3,SE3_b,SUR,SUR_b,Norte2,NTE2_b,NE3,NE3_b
0,O3,2020-01-01 00:00:00.000,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
1,O3,2020-01-01 01:00:00.000,19.0,,1.0,,19.0,,NaN,x,...,8.0,,25.0,,NaN,l,NaN,l,NaN,x
2,O3,2020-01-01 02:00:00.000,19.0,,1.0,,18.0,,NaN,x,...,10.0,,22.0,,NaN,l,NaN,l,NaN,x
3,O3,2020-01-01 03:00:00.000,19.0,,1.0,,14.0,,NaN,x,...,7.0,,21.0,,NaN,l,NaN,l,NaN,x
4,O3,2020-01-01 04:00:00.000,16.0,,NaN,s,5.0,,NaN,x,...,3.0,,22.0,,NaN,l,NaN,l,NaN,x


In [19]:
dfpadron = pd.read_excel("Padrón medio ambiente.xlsx")

In [20]:
dfpadron.head()

,MUNICIPIO,MARCA,LINEA,TIPO,MODELO,CILINDROS,COMBUSTIBLE,PESO,PROCEDENCIA,SERVICIO,CLASE_CLAVE_VEHICULAR,CANTIDAD
0,MONTERREY,NISSAN,TSURU VAGONETA,T/M TIPICA,1994,0,GASOLINA,0,ESTATAL,Particular,Automovil,1.0
1,SANTA CATARINA,FORD,F-150 PICK UP,XLT,1992,8,GASOLINA,2,IMPORTADOS USADOS,Particular,CAMION,5.0
2,SAN NICOLAS DE LOS GARZA,PEUGEOT,206,XS 1587,2005,4,GASOLINA,0,ESTATAL,Particular,Automovil,5.0
3,DR. ARROYO,FORD,EXPLORER,TRAC DOBLE,1996,6,GASOLINA,NaN,IMPORTADOS USADOS,Particular,Automovil,2.0
4,SAN NICOLAS DE LOS GARZA,CHEVROLET,SUBURBAN PAQ N,AUT A/ACON,1995,8,GASOLINA,0,ESTATAL,Particular,Automovil,10.0
